In [1]:
# generate info for station locations from reverse geocode
# basically taken from https://towardsdatascience.com/reverse-geocoding-with-nyc-bike-share-data-cdef427987f8/


In [2]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

In [3]:
geolocator = Nominatim(user_agent="citibikedata_analysis")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1, max_retries=2)

In [4]:
# check
reverse("40.741740, -73.994156").raw['address']

{'amenity': 'Citi Bike - West 21st Street & 6th Avenue',
 'road': 'West 21st Street',
 'neighbourhood': 'Chelsea',
 'borough': 'Manhattan',
 'county': 'New York County',
 'city': 'City of New York',
 'state': 'New York',
 'ISO3166-2-lvl4': 'US-NY',
 'postcode': '10011',
 'country': 'United States',
 'country_code': 'us'}

In [5]:
reverse("40.717548, -73.994156").raw['address']

{'house_number': '87',
 'road': 'Chrystie Street',
 'neighbourhood': 'Two Bridges',
 'borough': 'Manhattan',
 'county': 'New York County',
 'city': 'City of New York',
 'state': 'New York',
 'ISO3166-2-lvl4': 'US-NY',
 'postcode': '10002',
 'country': 'United States',
 'country_code': 'us'}

In [6]:
reverse("40.793367, -73.923956").raw['address']

{'road': 'Randalls Island Greenway',
 'neighbourhood': 'Manhattan Community Board 11',
 'borough': 'Manhattan',
 'county': 'New York County',
 'city': 'City of New York',
 'state': 'New York',
 'ISO3166-2-lvl4': 'US-NY',
 'postcode': '10454',
 'country': 'United States',
 'country_code': 'us'}

In [10]:
# import unique station names with coords
all_years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

imported_df_coords_per_year = {}
stations_analysis_list_all = []
for year in all_years:
    df = pd.read_csv('intermediate_results/df_coords_stations_' + str(year) + '.csv', index_col="Unnamed: 0")
    imported_df_coords_per_year[year] = df
    stations_analysis_list_all.append(df)

# make a single data frame with unique entries
df_all = pd.concat(stations_analysis_list_all, axis=0)
print(len(df_all))

# sum over index to get usage values for all stations
only_usage = pd.DataFrame(df_all["usage"].groupby(level=0).sum())
station_names_to_coords = dict(pd.Series(list(zip(df_all['longitude'], df_all['latitude'])),
                               index=df_all.index).to_dict())
df_coords = pd.DataFrame.from_dict(station_names_to_coords, orient='index')
df_coords = df_coords.rename(columns={0: 'longitude', 1: 'latitude'})
df_coords_all = df_coords.join(only_usage).sort_values('usage', ascending=False)
df_only_coords_all = df_coords_all.drop('usage', axis=1)

print(len(df_only_coords_all))
df_only_coords_all_cleaned = df_only_coords_all[df_only_coords_all['longitude'] != 0]
print(len(df_only_coords_all_cleaned))
df_only_coords_all_cleaned


13980
2756
2748


,longitude,latitude
W 21 St & 6 Ave,-73.994156,40.741740
West St & Chambers St,-74.013221,40.717548
E 17 St & Broadway,-73.990134,40.737006
Broadway & E 14 St,-73.990741,40.734546
8 Ave & W 31 St,-73.994685,40.750585
...,...,...
Forrest portable kiosk,-73.932004,40.703048
StuyTown Depot,-73.993859,40.759030
7 Ave & Bleecker St,-74.003638,40.732139
Sedgwick Ave & W Tremont Ave - new,-73.918306,40.853045


In [8]:
# call reverse function on those 
locations=[]
for index, row in df_only_coords_all_cleaned.iterrows():
    locations.append(reverse("{}, {}".format(row['latitude'],
    row['longitude'])).raw['address'])
df_geocode = pd.DataFrame(locations)
df_geocode.index = df_only_coords_all_cleaned.index

#save out to file
df_geocode.to_csv('intermediate_results/df_geocode_all_stations.csv', index=True)

df_geocode


RateLimiter caught an error, retrying (0/2 tries). Called with (*('40.70684203101325, -73.9544353965357',), **{}).
Traceback (most recent call last):
  File "C:\Users\tzach\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\Users\tzach\anaconda3\Lib\site-packages\urllib3\connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\tzach\anaconda3\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\Users\tzach\anaconda3\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\tzach\anaconda3\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\tzach\an

,amenity,road,neighbourhood,borough,county,city,state,ISO3166-2-lvl4,postcode,country,...,quarter,residential,highway,farm,industrial,hamlet,emergency,craft,town,region
W 21 St & 6 Ave,Citi Bike - West 21st Street & 6th Avenue,West 21st Street,Chelsea,Manhattan,New York County,City of New York,New York,US-NY,10011,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
West St & Chambers St,NaN,Chambers Street,Tribeca,Manhattan,New York County,City of New York,New York,US-NY,10282,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E 17 St & Broadway,NaN,East 17th Street,Union Square,Manhattan,New York County,City of New York,New York,US-NY,10003,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Broadway & E 14 St,Citi Bike - Broadway & East 14th Street,Broadway,University Village,Manhattan,New York County,City of New York,New York,US-NY,10003,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8 Ave & W 31 St,NaN,8th Avenue,Chelsea,Manhattan,New York County,City of New York,New York,US-NY,10199,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Forrest portable kiosk,NaN,Forrest Street,NaN,Brooklyn,Kings County,City of New York,New York,US-NY,11206,United States,...,Bushwick,NaN,NaN,NaN,NaN,NaN,NaN,Sweet Bella,NaN,NaN
StuyTown Depot,NaN,West 42nd Street,Hell's Kitchen,Manhattan,New York County,City of New York,New York,US-NY,10036,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7 Ave & Bleecker St,St. George Coffee,7th Avenue South,West Village,Manhattan,New York County,City of New York,New York,US-NY,10014,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sedgwick Ave & W Tremont Ave - new,NaN,Sedgwick Avenue,Morris Heights,The Bronx,Bronx County,City of New York,New York,US-NY,10453,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
